In [45]:
import pandas as pd
import numpy as np

# Adjust notebook settings to widen the notebook
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:95% !important;}</style>"))
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [46]:
hpsa = pd.read_csv("./Data/HPSA_Cleaned.csv")
nsduh = pd.read_csv("./Data/nsduh_data_cleaned.csv")
grants = pd.read_csv("./Data/grants_data_cleaned.csv")

In [47]:
hpsa.head(2)

,Withdrawn Date,HPSA Name,Common State County FIPS Code,Common State FIPS Code,Common State Name,County Equivalent Name,HPSA Component Name,HPSA Component Type Code,HPSA Component Type Description,HPSA Designation Population Type Description,Common State Abbreviation,HPSA Metropolitan Indicator Code,HPSA Type Code,Primary State FIPS Code,Primary State Name,State Abbreviation,State and County Federal Information Processing Standard Code,State FIPS Code,State Name,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,HPSA Status Code,Common Region Name,County or County Equivalent Federal Information Processing Standard Code,Common County Name,Metropolitan Indicator,HPSA Designation Last Update Date,HPSA Designation Date,HPSA Status,HPSA Geography Identification Number,HPSA Score,Primary State Abbreviation,Primary HHS Region Name,Designation Type,HPSA ID,HPSA Designation Population,Rural Status,Rural Status Code,HPSA Degree of Shortage,HPSA FTE,HPSA Component State Abbreviation,HPSA Population Type,HPSA Population Type Code,HPSA Shortage,DaysBeforeWithdrawn,County Equivalent Name New,countycountyequivalent,centraloutlyingcounty,statename,cbsacode,metropolitanmicropolitanstatis,PDEN10
0,NaN,Stanley Correctional Institution,55017,55,Wisconsin,Chippewa,Stanley Correctional Institution,UNK,Unknown,Correctional Facility,WI,0,PRSN,55,Wisconsin,WI,55017,55,Wisconsin,N,N,D,Region 5,017,"Chippewa County, WI",Unknown,08/02/2018,07/21/2003,Designated,POINT,15,WI,Region 5,Correctional Facility,7551065910,2885.0,Non-Rural,N,6,0.6,NaN,NaN,NaN,0.84,NaN,Chippewa County,Chippewa County,Central,Wisconsin,20740.0,Metropolitan Statistical Area,1
1,07/02/2018,Rock County,55105,55,Wisconsin,Rock,Rock,SCTY,Single County,Geographic Population,WI,0,Hpsa Geo HN,55,Wisconsin,WI,55105,55,Wisconsin,N,N,W,Region 5,105,"Rock County, WI",Unknown,07/02/2018,04/09/2014,Withdrawn,55105,13,WI,Region 5,High Needs Geographic HPSA,755105,156639.0,Partially Rural,P,Not applicable,8.5,WI,Geographic Population,TRC,0.74,1545 days,Rock County,Rock County,Central,Wisconsin,27500.0,Metropolitan Statistical Area,1


In [48]:
grants.head(2)

,Complete County Name,State Name,Total Active Grant Financial Assistance,Mental Health Assistance,countycountyequivalent,centraloutlyingcounty,statename,cbsacode,Outlying
0,Ada County,Idaho,9754823.0,460000.0,Ada County,Central,Idaho,14260.0,0
1,Adair County,Missouri,3364332.0,0.0,Adair County,Central,Missouri,28860.0,0


In [49]:
hpsa.shape

(27829, 52)

In [50]:
grants.shape

(711, 9)

In [51]:
# merge HPSA and Grant by CBSA identifiers
HPSA_Grant = hpsa.merge(grants, how='left', left_on=['countycountyequivalent', 'statename', 'cbsacode', 'State Name', 'centraloutlyingcounty' ], right_on=['countycountyequivalent', 'statename', 'cbsacode', 'State Name', 'centraloutlyingcounty'])

In [52]:
HPSA_Grant.columns

Index(['Withdrawn Date', 'HPSA Name', 'Common State County FIPS Code',
       'Common State FIPS Code', 'Common State Name', 'County Equivalent Name',
       'HPSA Component Name', 'HPSA Component Type Code',
       'HPSA Component Type Description',
       'HPSA Designation Population Type Description',
       'Common State Abbreviation', 'HPSA Metropolitan Indicator Code',
       'HPSA Type Code', 'Primary State FIPS Code', 'Primary State Name',
       'State Abbreviation',
       'State and County Federal Information Processing Standard Code',
       'State FIPS Code', 'State Name',
       'U.S. - Mexico Border 100 Kilometer Indicator',
       'U.S. - Mexico Border County Indicator', 'HPSA Status Code',
       'Common Region Name',
       'County or County Equivalent Federal Information Processing Standard Code',
       'Common County Name', 'Metropolitan Indicator',
       'HPSA Designation Last Update Date', 'HPSA Designation Date',
       'HPSA Status', 'HPSA Geography Identifica

In [53]:
# remove repetitive columns & not useful columns for Tableau visualizations
HPSA_Grant.drop(['Common State County FIPS Code', 
                 'Common State FIPS Code',
                 'Common State Name', 
                 'County Equivalent Name', 
                 'Common State Abbreviation',
                 'HPSA Metropolitan Indicator Code', 
                 'Primary State FIPS Code', 
                 'Primary State Name',
                 'State Abbreviation', 
                 'State FIPS Code', 
                 'State Name', 
                 'Common County Name', 
                 'Metropolitan Indicator',
                 'HPSA Status Code',  
                 'Rural Status Code',  
                 'HPSA Component State Abbreviation',
                 'HPSA Population Type Code', 
                 'County Equivalent Name New',
                 'Complete County Name',
                 'Outlying'], axis=1, inplace=True)
 
# rename columns names
HPSA_Grant.rename({
    'countycountyequivalent': 'County Name', 
    'statename': 'State Name',
    'cbsacode' : 'CBSA Code',
    'metropolitanmicropolitanstatis' : 'Metro or Micro'}, axis=1, inplace=True)

In [54]:
HPSA_Grant.columns

Index(['Withdrawn Date', 'HPSA Name', 'HPSA Component Name',
       'HPSA Component Type Code', 'HPSA Component Type Description',
       'HPSA Designation Population Type Description', 'HPSA Type Code',
       'State and County Federal Information Processing Standard Code',
       'U.S. - Mexico Border 100 Kilometer Indicator',
       'U.S. - Mexico Border County Indicator', 'Common Region Name',
       'County or County Equivalent Federal Information Processing Standard Code',
       'HPSA Designation Last Update Date', 'HPSA Designation Date',
       'HPSA Status', 'HPSA Geography Identification Number', 'HPSA Score',
       'Primary State Abbreviation', 'Primary HHS Region Name',
       'Designation Type', 'HPSA ID', 'HPSA Designation Population',
       'Rural Status', 'HPSA Degree of Shortage', 'HPSA FTE',
       'HPSA Population Type', 'HPSA Shortage', 'DaysBeforeWithdrawn',
       'County Name', 'centraloutlyingcounty', 'State Name', 'CBSA Code',
       'Metro or Micro', 'PDEN1

In [55]:
## save merged HPSA and Grant by CBSA columns dataset in ./Data folder 
HPSA_Grant.to_csv('./Data/hpsa_grant_cbsa.csv',index=False)

In [64]:
df = HPSA_Grant[[
    'PDEN10', 'Metro or Micro', 'County Name', 'State Name', 'Rural Status',
    'HPSA Name', 'HPSA Component Name',
    'HPSA Component Type Code', 'HPSA Component Type Description',
    'HPSA Designation Population Type Description', 'HPSA Type Code',
    'State and County Federal Information Processing Standard Code',
    'U.S. - Mexico Border 100 Kilometer Indicator',
    'U.S. - Mexico Border County Indicator', 'Common Region Name',
    'County or County Equivalent Federal Information Processing Standard Code',
    'HPSA Status', 'HPSA Geography Identification Number', 'HPSA Score',
    'Primary State Abbreviation', 'Primary HHS Region Name',
    'Designation Type', 'HPSA ID', 'HPSA Designation Population',
    'HPSA Degree of Shortage', 'HPSA FTE', 'HPSA Population Type',
    'HPSA Shortage', 'HPSA Designation Last Update Date', 'HPSA Designation Date',
    'Withdrawn Date', 'DaysBeforeWithdrawn',
    'CBSA Code', 'Total Active Grant Financial Assistance', 'Mental Health Assistance']]

In [65]:
df.head(3)

,PDEN10,Metro or Micro,County Name,State Name,Rural Status,HPSA Name,HPSA Component Name,HPSA Component Type Code,HPSA Component Type Description,HPSA Designation Population Type Description,HPSA Type Code,State and County Federal Information Processing Standard Code,U.S. - Mexico Border 100 Kilometer Indicator,U.S. - Mexico Border County Indicator,Common Region Name,County or County Equivalent Federal Information Processing Standard Code,HPSA Status,HPSA Geography Identification Number,HPSA Score,Primary State Abbreviation,Primary HHS Region Name,Designation Type,HPSA ID,HPSA Designation Population,HPSA Degree of Shortage,HPSA FTE,HPSA Population Type,HPSA Shortage,HPSA Designation Last Update Date,HPSA Designation Date,Withdrawn Date,DaysBeforeWithdrawn,CBSA Code,Total Active Grant Financial Assistance,Mental Health Assistance
0,1,Metropolitan Statistical Area,Chippewa County,Wisconsin,Non-Rural,Stanley Correctional Institution,Stanley Correctional Institution,UNK,Unknown,Correctional Facility,PRSN,55017,N,N,Region 5,017,Designated,POINT,15,WI,Region 5,Correctional Facility,7551065910,2885.0,6,0.6,NaN,0.84,08/02/2018,07/21/2003,NaN,NaN,20740.0,NaN,NaN
1,1,Metropolitan Statistical Area,Rock County,Wisconsin,Partially Rural,Rock County,Rock,SCTY,Single County,Geographic Population,Hpsa Geo HN,55105,N,N,Region 5,105,Withdrawn,55105,13,WI,Region 5,High Needs Geographic HPSA,755105,156639.0,Not applicable,8.5,Geographic Population,0.74,07/02/2018,04/09/2014,07/02/2018,1545 days,27500.0,86677.0,0.0
2,3,NaN,NaN,NaN,Rural,Polk County,Polk,SCTY,Single County,Geographic Population,Hpsa Geo HN,55095,N,N,Region 5,095,Withdrawn,55095,16,WI,Region 5,High Needs Geographic HPSA,755095,43071.0,Not applicable,3.9,Geographic Population,1.24,07/02/2018,12/20/2002,07/02/2018,5673 days,NaN,NaN,NaN


In [62]:
# df.groupby(['PDEN10'], as_index=False).agg(
#     {'County Name':'count',
#      'Total Active Grant Financial Assistance':'mean',
#     'State Name': 'count',
#     'Rural Status' : 'count'}
# )

,PDEN10,County Name,Total Active Grant Financial Assistance
0,1,19142,4.798134e+07
1,2,2647,3.846204e+06
2,3,0,NaN
